In [40]:
import numpy as np
import matplotlib.pyplot as plt 
import uproot
import glob, os
import sys
import re

# Code for reading files in order
numbers = re.compile(r'(\d+)')
def NumericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def ReadTheoreticalDataSRIM(material):
    filename = "{0}/stoppingPowerData/format_p_{1}_SRIM.txt".format(os.getcwd(),material)
    file = np.loadtxt(filename, skiprows=1)
    energy = file[:,0]  # mg to g/
    dEdxElec = file[:,1]/ 1e-03
    dEdxNuc = file[:,2]/ 1e-03
    dEdx = np.add(file[:,1],file[:,2]) / 1e-03 # Get sum of colluns 1 e 2 (of dE/dx)   
    return energy, dEdx, dEdxElec, dEdxNuc

def ReadTheoreticalDataNIST(material):
    filename = "{0}/stoppingPowerData/p_{1}.txt".format(os.getcwd(),material)
    file = np.loadtxt(filename, skiprows=8)
    energy = file[:,0]  
    dEdxElec = file[:,1]
    dEdxNuc = file[:,2]
    dEdx = file[:,3]  
    return energy, dEdx, dEdxElec, dEdxNuc

currentPath = os.getcwd()

materials = ["Si", "SiC", "C"]

for material in materials:
    eDepMean = []
    primaryEnergy = []
    fig2,ax2 = plt.subplots()
    ax2.set_title('{0} Stopping Power'.format(material))
    ax2.set_ylabel(r'$dE/dx$ (MeV/g/cm2)')
    ax2.set_xlabel(r'Primary Energy (MeV)')
    ax2.set_xlim(xmin=10, xmax=6e3)
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    axins = ax2.inset_axes([0.5, 0.5, 0.47, 0.47])
    x1, x2, y1, y2 = 900, 5000, 1.3, 2.6
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.tick_params(axis='both', labelsize=8)
    energy, dEdx, dEdxElec, dEdxNuc = ReadTheoreticalDataSRIM(material)
    axins.plot(energy, dEdx)
    plt.plot(energy, dEdx, label="{0} - SRIM 2013".format(material))
    if (material != "SiC"):
        energy, dEdx, dEdxElec, dEdxNuc = ReadTheoreticalDataNIST(material)
        plt.plot(energy, dEdx, label="{0} - NIST (PSTAR)".format(material))
        axins.plot(energy, dEdx)
    for filepath in sorted(glob.iglob("{0}/results/{1}/*".format(currentPath, material)), key=NumericalSort):
        file = uproot.open("{0}:Event".format(filepath))
        eDepMean.append(file["fdEdxPrimary"].array(library="np").mean())
        primaryEnergy.append(file["fPrimaryEnergy"].array(library="np").mean())
        
    ax2.scatter(primaryEnergy, eDepMean, label="{0} - Geant4 11.1.0".format(material))
    axins.errorbar(primaryEnergy, eDepMean, fmt='o')
    ax2.indicate_inset_zoom(axins, edgecolor="black")
    ax2.legend()
    ax2.figure.savefig("{0}-Validation.pdf".format(material))
    plt.close(fig2)

    